In [2]:
import pandas as pd
import numpy as np
from metrics import TCS
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = 'plotly_white'
plot_template = dict(
    layout=go.Layout({
        'font_size': 8,
        'xaxis_title_font_size': 8,
        'yaxis_title_font_size': 8,
        }   
))
hydro_stations = ['Tangnaihai','Guide','Xunhua']
hydrostation_abbrs = {'Tangnaihai':'TNH','Guide':'GD','Xunhua':'XH'}
hydrostation_channel = {'Tangnaihai':'3','Guide':'14','Xunhua':'9'}


Read simulation results of different models and evaluate the performance.


Read the prediction of different models and evaluate the performance.


In [12]:
pred_years = np.arange(2015,2020)
index = pd.date_range(start=f'{pred_years[0]}-01-01',end=f'{pred_years[-1]}-12-31',freq='MS')
for hydro_station in hydro_stations:
    hydro_station_abbr = hydrostation_abbrs[hydro_station]
    hydro_station_channel = hydrostation_channel[hydro_station]
    
    obs_df = pd.read_csv(f'../data/{hydro_station.lower()}_natural_monthly_flow.csv',index_col=['date'],parse_dates=['date'])
    obs_his = obs_df.copy().loc[:f'{pred_years[0]-1}-12-31']
    obs_his_monthly_mean = obs_his.groupby(obs_his.index.month)['flow(m^3/s)'].mean()
    # Create a new DataFrame with monthly average flow for 2015
    df_monthly_avg = pd.DataFrame(index=index)
    # Replicate the monthly average flow for years 2015-2019
    for year in range(pred_years[0],pred_years[-1]+1):
        for month in range(1,13):   
            df_monthly_avg.loc[f'{year}-{month}-01','flow(m^3/s)'] = obs_his_monthly_mean[month]

    arima_pred = pd.read_csv(f'../result/ARIMAPredData/seasonal_decompose_multiplicative_arima_pred_{hydro_station.lower()}_2015_{pred_years[-1]}.csv',index_col=['date'],parse_dates=['date'])
    SWATPlus_simu = pd.read_csv(f'../result/SWATPlusCalValSimData/Channel_{hydro_station_channel}_Monthly_River-Flow_{hydro_station}_Sim1972_2019.csv',index_col=['Date'],parse_dates=['Date'])
    SWATPlus_simu = SWATPlus_simu.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31',:]
    SWATPlus_simu = SWATPlus_simu.sort_index()
    SWATPlus_simu.index = index
    
    pred_data_swatp_mete_simyr = pd.read_csv(f'../result/SWATPlusPredUsingMeteSimYearData/{hydro_station}_SWATPlus_pred_obs_{pred_years[0]}_{pred_years[-1]}.csv',index_col=['date'],parse_dates=['date'])
    pred_data_swatp_hydro_simyr = pd.read_csv(f'../result/SWATPlusPredUsingHydroSimYearData/{hydro_station}_SWATPlus_pred_obs_{pred_years[0]}_{pred_years[-1]}.csv',index_col=['date'],parse_dates=['date'])
    pred_data_swatp_clip_ssp126 = pd.read_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_ssp126_{pred_years[0]}_{pred_years[-1]}_corrected_using_historical_mean_std.csv',index_col=['date'],parse_dates=['date'])
    pred_data_swatp_clip_ssp245 = pd.read_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_ssp245_{pred_years[0]}_{pred_years[-1]}_corrected_using_historical_mean_std.csv',index_col=['date'],parse_dates=['date'])
    pred_data_swatp_clip_ssp585 = pd.read_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_ssp585_{pred_years[0]}_{pred_years[-1]}_corrected_using_historical_mean_std.csv',index_col=['date'],parse_dates=['date'])      
    
    
    print('R2,TSC for average monthly flow:',
          r2_score(obs_df.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)'],df_monthly_avg['flow(m^3/s)']),
          TCS(obs_df.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)'],df_monthly_avg['flow(m^3/s)']))
    print('R2,TCS for SARIMA:',
          r2_score(obs_df.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)'],arima_pred.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)']),
          TCS(obs_df.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)'],arima_pred.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)']))
    print('R2,TCS for SWATPlus Mete SimYr:',
          r2_score(pred_data_swatp_mete_simyr['obs'],pred_data_swatp_mete_simyr['pred']),
          TCS(pred_data_swatp_mete_simyr['obs'],pred_data_swatp_mete_simyr['pred']))
    print('R2,TCS for SWATPlus Hydro SimYr:',
          r2_score(pred_data_swatp_hydro_simyr['obs'],pred_data_swatp_hydro_simyr['pred']),
          TCS(pred_data_swatp_hydro_simyr['obs'],pred_data_swatp_hydro_simyr['pred']))
    print('R2,TCS for SWATPlus SSP126:',
          r2_score(pred_data_swatp_clip_ssp126['obs'],pred_data_swatp_clip_ssp126['corrected_pred']),
          TCS(pred_data_swatp_clip_ssp126['obs'],pred_data_swatp_clip_ssp126['corrected_pred']))
    print('R2,TCS for SWATPlus SSP245:',
          r2_score(pred_data_swatp_clip_ssp245['obs'],pred_data_swatp_clip_ssp245['corrected_pred']),
          TCS(pred_data_swatp_clip_ssp245['obs'],pred_data_swatp_clip_ssp245['corrected_pred']))
    print('R2,TCS for SWATPlus SSP585:',
          r2_score(pred_data_swatp_clip_ssp585['obs'],pred_data_swatp_clip_ssp585['corrected_pred']),
          TCS(pred_data_swatp_clip_ssp585['obs'],pred_data_swatp_clip_ssp585['corrected_pred']))
    # visualize
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=obs_df.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31'].index, y=obs_df.loc[f'{pred_years[0]}-01-01':f'{pred_years[-1]}-12-31','flow(m^3/s)'], mode='lines', name='Observed', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=SWATPlus_simu.index,y=SWATPlus_simu['Value'],mode='lines',name='SWATPlus Simulated',line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df_monthly_avg.index,y=df_monthly_avg['flow(m^3/s)'],mode='lines',name='Average Monthly Flow',line=dict(color='black')))
    fig.add_trace(go.Scatter(x=arima_pred.index,y=arima_pred['flow(m^3/s)'],mode='lines',name='SARIMA Pred',line=dict(color='tomato')))
    fig.add_trace(go.Scatter(x=pred_data_swatp_mete_simyr.index,y=pred_data_swatp_mete_simyr['pred'],mode='lines',name='SWATPlus Mete SimYr',line=dict(color='purple')))
    fig.add_trace(go.Scatter(x=pred_data_swatp_hydro_simyr.index,y=pred_data_swatp_hydro_simyr['pred'],mode='lines',name='SWATPlus Hydro SimYr',line=dict(color='green')))
    fig.add_trace(go.Scatter(x=pred_data_swatp_clip_ssp126.index,y=pred_data_swatp_clip_ssp126['corrected_pred'],mode='lines',name='SWATPlus SSP126',line=dict(color='lightblue')))
    fig.add_trace(go.Scatter(x=pred_data_swatp_clip_ssp245.index,y=pred_data_swatp_clip_ssp245['corrected_pred'],mode='lines',name='SWATPlus SSP245',line=dict(color='cyan')))
    fig.add_trace(go.Scatter(x=pred_data_swatp_clip_ssp585.index,y=pred_data_swatp_clip_ssp585['corrected_pred'],mode='lines',name='SWATPlus SSP585',line=dict(color='pink')))
    fig.show()


R2,TSC for average monthly flow: 0.5820167087622052 0.7627118644067796
R2,TCS for SARIMA: 0.7699014706302801 0.7288135593220338
R2,TCS for SWATPlus Mete SimYr: 0.4466293411687805 0.4915254237288136
R2,TCS for SWATPlus Hydro SimYr: 0.4046050365242787 0.288135593220339
R2,TCS for SWATPlus SSP126: 0.01663569197310566 0.22033898305084745
R2,TCS for SWATPlus SSP245: 0.3376623819092923 0.288135593220339
R2,TCS for SWATPlus SSP585: 0.33931537401980294 0.22033898305084745


R2,TSC for average monthly flow: 0.5905394097251069 0.7966101694915254
R2,TCS for SARIMA: 0.7158223710906776 0.3898305084745763
R2,TCS for SWATPlus Mete SimYr: 0.5565269716805198 0.2542372881355932
R2,TCS for SWATPlus Hydro SimYr: 0.3927137481280527 0.22033898305084745
R2,TCS for SWATPlus SSP126: 0.31455564709053374 0.15254237288135594
R2,TCS for SWATPlus SSP245: 0.28378951111885 0.1864406779661017
R2,TCS for SWATPlus SSP585: 0.27894797403474003 0.1864406779661017


R2,TSC for average monthly flow: 0.5973779505176704 0.8305084745762712
R2,TCS for SARIMA: 0.7112338620142227 0.7288135593220338
R2,TCS for SWATPlus Mete SimYr: 0.47982633075155046 0.3559322033898305
R2,TCS for SWATPlus Hydro SimYr: 0.4294475766419775 0.4576271186440678
R2,TCS for SWATPlus SSP126: 0.265913544949803 0.288135593220339
R2,TCS for SWATPlus SSP245: 0.25670290318369626 0.3220338983050847
R2,TCS for SWATPlus SSP585: 0.23437999771415208 0.288135593220339
